In [ ]:
import numpy as np
from scipy.stats import t, multivariate_normal
import matplotlib.pyplot as plt
import seaborn as sns

# --- Settings ---
df = 3               # degrees of freedom for Student-t
n_samples = 5000       # number of samples
dim = 2                # dimension (can increase if needed)
rho = -0.9             # strong negative correlation

# --- i.i.d. Student-t samples ---
iid_samples = t.rvs(df=df, size=(n_samples, dim))

# --- Negatively correlated Student-t samples ---
# Step 1: draw from multivariate normal with negative correlation
cov = np.full((dim, dim), rho)
np.fill_diagonal(cov, 1.0)
mean = np.zeros(dim)
Z = multivariate_normal.rvs(mean=mean, cov=cov, size=n_samples)

# Step 2: scale by chi-squared to get multivariate t-distribution
chi2_samples = np.random.chisquare(df, size=n_samples).reshape(-1, 1)
scaling = np.sqrt(df / chi2_samples)
correlated_samples = Z * scaling  # element-wise

# --- Optional: visualize KDE heatmaps ---
sns.set(style="whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.kdeplot(
    x=iid_samples[:, 0],
    y=iid_samples[:, 1],
    ax=axes[0],
    fill=True,
    cmap="Blues",
    levels=10  # <-- add this line
)

axes[0].set_title("i.i.d. Student-t samples (df=3)")
axes[0].set_xlim(-10, 10)
axes[0].set_ylim(-10, 10)
sns.kdeplot(
    x=correlated_samples[:, 0],
    y=correlated_samples[:, 1],
    ax=axes[1],
    fill=True,
    cmap="Reds",
    levels=10  # <-- add this line
)

axes[1].set_title("Negatively correlated Student-t samples (df=3)")
axes[1].set_xlim(-10, 10)
axes[1].set_ylim(-10, 10)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, norm

# Settings
x = np.linspace(1, 20, 500)  # focus on tails
df = 3

# Analytical survival functions (two-sided tails)
t_tail = 2 * t.sf(x, df=df)
gaussian_tail = 2 * norm.sf(x)

# Compare with theoretical bound from (H3): c / x
c2 = 1.0
H3_bound = c2 / x

# Plotting
plt.figure(figsize=(8, 5))
plt.plot(x, t_tail, label="Student-t (df=1.5)", lw=2)
plt.plot(x, gaussian_tail, label="Gaussian", lw=2, linestyle="--")
plt.plot(x, H3_bound, label=r"Theoretical bound $c_2/x$", lw=2, linestyle=":")
plt.yscale("log")
plt.xlabel(r"$x$")
plt.ylabel(r"$P(|Z| > x)$ (log scale)")
plt.title("Tail decay comparison")
plt.legend()
plt.grid(True, which="both", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t

# Settings
x = np.linspace(1e-6, 20, 500)  # avoid log(0)
df = 3
kappa = 1.0  # Can be 0.5, 1.0, etc.

# Compute density and log(1/h(x))
h_x = t.pdf(x, df=df)
log_inv_hx = np.log(1 / h_x)

c_1 = 2
# RHS of H2: c1 * (1 + log^{1+kappa}(1 + x))
log_growth = c_1*(1 + np.log(1 + x) ** (1 + kappa))

# Plotting
plt.figure(figsize=(8, 5))
plt.plot(x, log_inv_hx, label=r"$\log(1/h(x))$", lw=2)
plt.plot(x, log_growth, label=rf"$\log^{{1+\kappa}}(1 + x)$ with $\kappa = {kappa}$", lw=2, linestyle="--")
plt.xlabel(r"$x$")
plt.ylabel("log-density growth")
plt.title("Verifying H2: log(1/h(x)) vs log^{1+κ}(1 + x)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t, multivariate_normal, gaussian_kde

# Settings
df = 3
kappa = 1.0
c_1 = 2
n_samples = 1_000_000
dim = 2
rho = -0.9
x = np.linspace(1e-6, 20, 500)

# --- Theoretical log(1/h(x)) and H2 upper bound ---
h_x = t.pdf(x, df=df)
log_inv_hx = np.log(1 / h_x)
log_growth = c_1 * (1 + np.log(1 + x) ** (1 + kappa))

# --- Simulate i.i.d. Student-t samples ---
iid_samples = t.rvs(df=df, size=(n_samples, dim))

# --- Simulate correlated Student-t samples ---
cov = np.full((dim, dim), rho)
np.fill_diagonal(cov, 1.0)
Z = multivariate_normal.rvs(mean=np.zeros(dim), cov=cov, size=n_samples)
chi2 = np.random.chisquare(df, size=n_samples).reshape(-1, 1)
scaling = np.sqrt(df / chi2)
correlated_samples = Z * scaling

# --- Compute empirical KDE-based log(1/h(x)) for marginals ---
dim_to_plot = 0  # marginal to use
iid_kde = gaussian_kde(np.abs(iid_samples[:, dim_to_plot]))
corr_kde = gaussian_kde(np.abs(correlated_samples[:, dim_to_plot]))
emp_log_inv_iid = np.log(1 / iid_kde(x))
emp_log_inv_corr = np.log(1 / corr_kde(x))

# --- Plot all in one ---
plt.figure(figsize=(10, 6))
plt.plot(x, log_inv_hx, label=r"Theoretical $\log(1/h(x))$", lw=2, color="black")
plt.plot(x, log_growth, label=rf"H2 bound: $c_1(1 + \log^{{1+\kappa}}(1 + x))$", lw=2, linestyle="--", color="gray")
plt.plot(x, emp_log_inv_iid, label="Empirical log(1/h(x)), i.i.d.", lw=2, color="blue")
plt.plot(x, emp_log_inv_corr, label="Empirical log(1/h(x)), correlated", lw=2, color="red")

plt.xlabel(r"$x$")
plt.ylabel(r"$\log(1/h(x))$")
plt.title(r"Verifying H2: Theoretical and Empirical $\log(1/h(x))$ for $t_\nu$, $\nu=3$")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import dirichlet, t

# Parameters
n_obs = 50
n = 1000                      # number of samples
d = 10000                         # dimensionality of Dirichlet (choose one weight only)
tau = 0.1
nu = 3                          # degrees of freedom for Student-t
clip = 10                       # clip Student-t to [-10, 10]

# Step 1: Draw Dirichlet weights
alpha = np.random.dirichlet([tau] * d, size=n)[:, 0]  # take first component

# Step 2: Draw Student-t and clip
t_samples = np.clip(t.rvs(df=nu, size=n), -clip, clip)

# Step 3: Form w_j = alpha_j * t_j
w = alpha * t_samples

# Step 4: Compute a_n
logn = np.log(n_obs)
a_n = np.exp(-(logn)**2)

# Step 5: Plot histogram and a_n
plt.figure(figsize=(10, 6))
plt.hist(np.abs(w), bins=100, density=True, alpha=0.7, label='|w_j| = |alpha * t|')
plt.axvline(a_n, color='red', linestyle='--', label=f'$a_n = e^{{-(\log n)^2}}$ ≈ {a_n:.1e}')
plt.title(f'Empirical |w_j| distribution with τ={tau}, ν={nu}, n={n}')
plt.xlabel('|w_j|')
plt.ylabel('Density')
plt.legend()

# Step 6: Compute and print tail mass
tail_prob = np.mean(np.abs(w) > a_n)
print(f"Empirical P(|w_j| > a_n): {tail_prob:.6f}")

plt.show()


Tilhørende artikkel: https://arxiv.org/pdf/2506.19144

Tror nok dessverre sigma ikke vil la seg bli så forferdelig liten som det kreves for å oppnå disse minimax ratene.